## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli
# Trabajo práctico  3: Modelos de clasificación 

## Introducción y objetivo

El objetivo de este trabajo es predecir utilizando modelos de clasificacion si un cliente se dará de baja o no de la plataforma.

In [ ]:
#Las librerías utilizadas en este documento son:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import binarize

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
import seaborn as sns
sns.set()

import statsmodels.api as sm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


## Carga de datos

In [ ]:
data  = pd.read_csv('./data/Datos ML 2021 Q2.csv', sep=';')
print("El dataset tiene {} filas y {} columnas".format(data.shape[0], data.shape[1]))
data.head()

## Descripción del dataset

#### El dataset tiene las siguientes columnas:

- CustomerID: ID del cliente
- Churn: Columna que indica si el cliente dejó de usar la plataforma o no. 1 es que se da de baja.
- CustomerTenure: Es el tiempo transcurrido desde el inicio de la relación con el cliente (en meses)
- MainDeviceLogin: Dispositivo principal que utiliza el cliente para acceder a la plataforma
- CityTier: Indicador del nivel de desarrollo de la ciudad donde vive el cliente
- WarehouseToHome: Distancia desde el centro de distribución a la vivienda del cliente (en km)
- MainPaymentMode: Método de pago más utilizado por el cliente
- Gender: Género del cliente
- HourSpendOnApp: Número de horas que el cliente ha pasado en la plataforma
- DeviceRegistered: Número de dispositivos en los que el cliente ha accedido a la plataforma
- PrefCategory: Categoría más común de las compras del cliente en el último mes
- SatisfactionScore: Nivel de satisfacción del cliente con el servicio
- MaritalStatus: Estado civil del cliente
- NumberOfAddress: Número de direcciones diferentes registradas por el cliente
- Complain: Si ha realizado reclamos
- OrderAmountHikeFromlastYear: Incremento porcentual en la cantidad de compras con respecto al año anterior
- CouponUsed: Número de cupones usados en el último mes
- OrderCount: Número de compras realizadas en el último mes
- DaySinceLastOrder: Cantidad de días desde la última compra
- CashbackAmount: Promedio de reembolsos pedidos en el último mes

Variable objetivo: Churn (termino empleado en marketing para hacer referencia a si un cliente deja de usar una aplicación y/o regresa)

## Exploración de datos

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
display(data['Churn'].value_counts())
display(data['Churn'].value_counts(normalize=True))

### Detección de datos sospechosas o atípicos

In [ ]:
##Se consideran columnas con valores sospechosos aquellas cuya máxima valor se encuentran por encima de 3 desviaciones estándar de la media. 
std_limit = 3
##Por la naturaleza de las variables, se considera que los valores sospechosos son aquellos que se encuentran por encima y no los inferiores.

In [ ]:
# Columnas sospechosas

suspicious_columns = []

for col in data.columns:
    if(data[col].dtype == 'object'):
        continue
    mean = data[col].mean()
    std = data[col].std()
    max = data[col].max()
    if(max > mean + std_limit*std):
        suspicious_columns.append(data[col].name)
suspicious_columns


In [ ]:
suspicious_rows_arr = []

def investigate_suspicious_column(data, column, watch_outliers=True):
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    plt.suptitle(column)
    sns.histplot(data[column], ax=ax[0])
    sns.boxplot(data=data[column], ax=ax[1], orient='h')

    plt.show()

    if(watch_outliers):
        mean = data[column].mean()
        std = data[column].std()
        max = data[column].max()

        suspicious_rows = data[data[column] > mean + std_limit*std]
        suspicious_rows_arr.append(suspicious_rows)
        display("Hay {} filas sospechosas".format(suspicious_rows.shape[0]))
        display(suspicious_rows)

In [ ]:
for col in suspicious_columns:
    investigate_suspicious_column(data, col)

In [ ]:
## Total de filas sospechosas
print("Hay {} filas sospechosas".format(sum([suspicious_rows.shape[0] for suspicious_rows in suspicious_rows_arr])))

In [ ]:
## Filas sospechosas agrupadas por columna churn

suspicious_rows = pd.concat(suspicious_rows_arr)
display(suspicious_rows['Churn'].value_counts())
display(suspicious_rows['Churn'].value_counts(normalize=True))


Observamos que la distribución de la variable Churn entre los valores extremos es similar, por lo que no parece haber una relación entre las filas con estos datos y la variable objetivo.

Por otro lado, haciendo una observación pormenorizada, creemos que en los casos de las columnas `CouponUsed`, `OrderCount` y `DaySinceLastOrder`  y `CashbackAmount` parecen ser valores lógicos, aún tratándose de valores extremos por lo que no las eliminaremos del dataset original.

In [ ]:
suspicious_columns

Removemos las columnas mencionadas de la lista de columnas sospechosas

In [ ]:
suspicious_columns.remove('CouponUsed')
suspicious_columns.remove('OrderCount')
suspicious_columns.remove('DaySinceLastOrder')
suspicious_columns.remove('CashbackAmount')

suspicious_columns

Limpiemos las filas sospechosas de las columnas que quedaron como sospechosas:

CustomerTenure, WarehouseToHome, NumberOfAddress

In [ ]:
def remove_outliers(data, column):
    mean = data[column].mean()
    std = data[column].std()
    max = data[column].max()
    return data[data[column] <= mean + std_limit*std]

In [ ]:
for col in suspicious_columns:
    data = remove_outliers(data, col)

Veamos el resultado

In [ ]:
for col in suspicious_columns:
    investigate_suspicious_column(data, col, False)

### Correlación de las variables

PENDIENTE- PAIR PLOT

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(32,32))
fig.suptitle('Histogramas normalizados')
for c, ax in zip(data.columns, axes.flatten()):
    sns.histplot(data = data.loc[data['Churn']==0, c].dropna(), stat = 'density', ax = ax, kde = False )
    sns.histplot(data = data.loc[data['Churn']==1, c].dropna(), stat = 'density', kde=False, ax=ax, color = 'orange')
    ax.legend(['Churn = 0', 'Churn = 1'])

In [ ]:
sns.pairplot(data, hue='Churn')

PENDIENTE -Analizar pairplot

Me está diciendo que no hay correlación entre las variables, pero no estoy seguro de que sea así.

excepción sería la correlación entre `OrderCount` y `CouponUsed` 

Podemos inferir que las variables que tendrán una significancia en establecer el valor del churn son aquellas en las cuales podemos observar diferencias en la distribución de los valores de churn.

Por lo tanto podemos descartar las variables que no presentan diferencias en la distribución de los valores de churn.

De los graficos podemos observar que: CustomerID, Gender, NumberOfAddress son las columnas que no permiten diferenciar lo mencionado anteriormente

In [ ]:
## Variables a eliminar
columns_to_eliminate = ['CustomerID', 'Gender', 'NumberOfAddress']

data = data.drop(columns_to_eliminate, axis=1)

Veamos la correlación entre las variables

PENDIENTE- buscar correlación entre variables

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(data.corr(), annot=True, vmin=-1, cmap='Blues')

In [ ]:
## Correlación entre variables y churn
abs_corr = data.corr()[['Churn']].abs().sort_values(by='Churn', ascending=False)

In [ ]:
## Correlación entre variables y churn, absoluta (para ordenar sin tener en cuenta si es positiva o negativa)

plt.figure(figsize=(8,12))
sns.heatmap(abs_corr, annot=True)

Nos quedamos con las variables que tienen una correlación mayor a 0.1

In [ ]:
## Variables con correlación mayor a 0.1

high_corr_vars = abs_corr[abs_corr['Churn'] > 0.1]

## Lo convertimos en una lista para poder iterar sobre ella

high_corr_vars = high_corr_vars.index.tolist()

high_corr_vars

In [ ]:
# Modificamos nuestro dataset con las columnas de la lista que tienen correlación mayor a 0.1
data = data[high_corr_vars]

### Eliminación de valores nan

In [ ]:
## Veamos cuantos valores nulos hay en cada columna

data.isna().sum()

In [ ]:
## Los eliminamos

data.dropna(inplace=True)

display(data.isna().sum())

print("El dataset limpio tiene {} filas y {} columnas".format(data.shape[0], data.shape[1]))



Observemos la distribución de los valores de churn

In [ ]:
display(data['Churn'].value_counts())
display(data['Churn'].value_counts(normalize=True))

El dataset está desbalanceado, por lo que se deberá tener en cuenta al momento de entrenar los modelos.

## Separación de datos

PENDIENTE - estratificar

In [ ]:
X = data.drop(['Churn'], axis=1)
y = data['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Verificamos que la proporción de churn sea similar en los conjuntos de entrenamiento y testeo

print("Churn en el conjunto de entrenamiento")
display(y_train.value_counts(normalize=True))
print("Churn en el conjunto de testeo")
display(y_test.value_counts(normalize=True))


## Preparación de datos

In [ ]:
categorical_columns = [col for col in data.columns if data[col].dtypes == 'object']

categorical_columns

In [ ]:
numerical_columns = [col for col in data.columns if data[col].dtypes != 'object']

numerical_columns

In [ ]:
# DeviceRegistered es una variable categórica 

numerical_columns.remove('DeviceRegistered')
categorical_columns.append('DeviceRegistered')

# La transformamos a categórica

X_train['DeviceRegistered'] = X_train['DeviceRegistered'].astype('object')
X_test['DeviceRegistered'] = X_test['DeviceRegistered'].astype('object')


In [ ]:
# Complain es una variable categórica binaria 

numerical_columns.remove('Complain')
categorical_columns.append('Complain')

# La transformamos a categórica

X_train['Complain'] = X_train['Complain'].astype('object')
X_test['Complain'] = X_test['Complain'].astype('object')

In [ ]:
display('categorical_columns',categorical_columns)
display('numerical_columns',numerical_columns)

### Variables categóricas

In [ ]:
encoder_categories = []

X_categorical_columns = [x for x in categorical_columns]

for col in X_categorical_columns:    
    col_categories = data[col].unique()
    encoder_categories.append(col_categories)

encoder_categories

In [ ]:
encoder = OneHotEncoder(categories = encoder_categories, sparse=False, drop='first')

encoder = encoder.fit(X_train[X_categorical_columns])

X_train_encoded = encoder.transform(X_train[X_categorical_columns])
X_train_categorical = pd.DataFrame(X_train_encoded, columns = encoder.get_feature_names_out(X_categorical_columns))

X_test_encoded = encoder.transform(X_test[X_categorical_columns])
X_test_categorical = pd.DataFrame(X_test_encoded, columns = encoder.get_feature_names_out(X_categorical_columns))
X_test_categorical.head()

### Variables numéricas

In [ ]:
X_train_numerical = X_train.drop(X_categorical_columns, axis=1)
X_test_numerical = X_test.drop(X_categorical_columns, axis=1)

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_numerical)
X_train_numerical = pd.DataFrame(X_train_scaled, columns = X_train_numerical.columns)

X_test_scaled = scaler.transform(X_test_numerical)
X_test_numerical = pd.DataFrame(X_test_scaled, columns = X_test_numerical.columns)
X_test_numerical.head()

Unimos las variables numéricas y categóricas

In [ ]:
X_train = pd.concat([X_train_categorical, X_train_numerical], axis=1)
X_test = pd.concat([X_test_categorical, X_test_numerical], axis=1)

PENDIENTE- Probar con otro DF, con alguna variable transformada. 

PENDIENTE- explicar porque se eligen esas dos variables

In [ ]:
X_train = X_train[['CustomerTenure', 'Complain_0']]
X_test = X_test[['CustomerTenure', 'Complain_0']]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train.head()

In [ ]:
# null accuracy - accuracy predicha por un modelo que predice siempre la clase mayoritaria

total = y_test.shape[0]
tn = y_test.value_counts()[0]
fn = y_test.value_counts()[1]
tp = 0
fp = 0
null_accuracy = (tp + tn)/(tp + tn + fp + fn)
print("TN: {}".format(tn))
print("FN: {}".format(fn))
print("Null accuracy: {}".format(null_accuracy))

# También podemos calcularlo con la siguiente función
# y_test.value_counts(normalize=True).max()

## Utils para modelos

In [ ]:
def create_confusion_matrix(y_test, y_pred):
    
    conf_mat = confusion_matrix(y_test, y_pred)
    conf_mat_df = pd.DataFrame(conf_mat, index = ['Negative (No Churn)', 'Positive (Churn)'], columns = ['Negative (No Churn)', 'Positive (Churn)'])
    plt.figure(figsize=(5.5,4))
    sns.heatmap(conf_mat_df, annot=True, fmt='g', cmap='Blues')
    plt.title('Matriz de confusión')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.show()

In [ ]:
def create_metrics(y_test, y_pred):
    tn = confusion_matrix(y_test, y_pred)[0,0]
    fp = confusion_matrix(y_test, y_pred)[0,1]


    # accuracy = (tp + tn) / (tp + tn + fp + fn)

    print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

    # recall = tp / (tp + fn)

    print('Recall: %.3f' % recall_score(y_test, y_pred))

    # precision = tp / (tp + fp)

    print('Precision: %.3f' % precision_score(y_test, y_pred))

    # specificity = tn / (tn + fp)

    print('Specificity: %.3f' % (tn / (tn + fp)))

    # f1 = 2 * (precision * recall) / (precision + recall)

    print('F1 score: %.3f' % f1_score(y_test, y_pred))

In [ ]:
def create_roc_curve(y_test, y_pred_proba):
    
    fpr_log,tpr_log,thr_log = roc_curve(y_test, y_pred_proba[:,1])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_log, tpr_log, label='GNB')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('GNB ROC Curve')
    plt.show()

    # AUC - Area Under the Curve

    auc = roc_auc_score(y_test, y_pred_proba[:,1])
    print('AUC: %.2f' % auc)

In [ ]:
def compare_thresholds(y_test, y_pred, y_pred_proba):
    tn = confusion_matrix(y_test, y_pred)[0,0]
    fp = confusion_matrix(y_test, y_pred)[0,1]

    thresholds = np.arange(0, 1, 0.01)
    accuracy = []
    recall = []
    precision = []
    specificity = []
    f1 = []
    for i in thresholds:
        y_pred = binarize(y_pred_proba, threshold=i)[:,1]
        accuracy.append(accuracy_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        precision.append(precision_score(y_test, y_pred, zero_division=0))
        specificity.append(tn / (tn + fp))
        f1.append(f1_score(y_test, y_pred))
    plt.plot(thresholds, accuracy, label='accuracy')
    plt.plot(thresholds, recall, label='recall')
    plt.plot(thresholds, precision, label='precision')
    plt.plot(thresholds, specificity, label='specificity')
    plt.plot(thresholds, f1, label='f1')
    plt.legend()
    plt.xlabel('threshold')
    plt.ylabel('score')
    plt.show()

## Naive Bayes

### Modelo

In [ ]:
gnb = GaussianNB()

gnb.fit(X_train, y_train)

In [ ]:
y_pred_nb = gnb.predict(X_test)


In [ ]:
y_pred_proba_nb = gnb.predict_proba(X_test)

## KNN

### Modelo

In [ ]:
# Utilizamos grid search para encontrar los mejores parámetros

KNN = KNeighborsClassifier()

k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
print(param_grid)

folds=StratifiedKFold(n_splits=10, random_state=19, shuffle=True)

grid = GridSearchCV(KNN, param_grid, cv=folds, scoring='recall')

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
y_pred_knn = grid.predict(X_test)

In [ ]:
y_pred_proba_knn = grid.predict_proba(X_test)

## Regresión logística

### Modelo

In [ ]:
# Utilizamos grid search para encontrar los mejores parámetros

lr = LogisticRegression()

c_range = np.logspace(-2, 4, 7)

param_grid = dict(C=c_range)

grid = GridSearchCV(lr, param_grid, cv=folds, scoring='recall')

grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
y_pred_lr = grid.predict(X_test)

In [ ]:
y_pred_proba_lr = grid.predict_proba(X_test)

## Métricas

In [ ]:
print("Naive Bayes:")
create_confusion_matrix(y_test, y_pred_nb)
print("KNN:")
create_confusion_matrix(y_test, y_pred_knn)
print("Logistic Regression:")
create_confusion_matrix(y_test, y_pred_lr)

In [ ]:
print("Naive Bayes:")
create_metrics(y_test, y_pred_nb)
print("-------------")
print("KNN:")
create_metrics(y_test, y_pred_knn)
print("-------------")
print("Logistic Regression:")
create_metrics(y_test, y_pred_lr)

In [ ]:
print("Naive Bayes:")
create_roc_curve(y_test, y_pred_proba_nb)
create_roc_curve(y_test, y_pred_proba_knn)
create_roc_curve(y_test, y_pred_proba_lr)

PENDIENTE - hacer en un mismo grafico las tres metricas de recall- explicar xq se usa recall

In [ ]:
compare_thresholds(y_test, y_pred_nb, y_pred_proba_nb)
compare_thresholds(y_test, y_pred_knn, y_pred_proba_knn)
compare_thresholds(y_test, y_pred_lr, y_pred_proba_lr)

## Conclusiones - PENDIENTE

- Los datos modelados tienen un 85% de casos con clientes que se no abandonan la app y 15 % de clientes que se dan de baja, por lo cual, consideramos que representa un alto sesgo para el modelo.